In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.Trainer.Trainer import Trainer
from src.Models.Model_class import Model_class

from src.Models.Autoenc_simpl import Autoencoder
from src.Models.Classifier import Simple_classifier

from src.DatasetsClasses.EncDataset import EncoderDataset
from src.Losses.Loss_class_encoder import Loss_class_encoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
df = pd.read_csv('../data/df_to_enc.csv')
df = df.drop(columns = ['Machine failure'])

In [3]:
df.shape

(10000, 60)

In [4]:
HIDDEN_PARAM = 128
LATENT_REPR = 10

BATCH_SIZE = 1024

In [5]:
X_train, X_test = train_test_split(df, shuffle=True, random_state=42)

In [6]:
train_dataset = EncoderDataset(X_train)
val_dataset = EncoderDataset(X_test)

In [7]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [9]:
autoencoder = Autoencoder(df.shape[1], HIDDEN_PARAM, LATENT_REPR)

In [10]:
loss = Loss_class_encoder(nn.MSELoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=autoencoder,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'Autoencoder_simple_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [11]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/autoencoder/'
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [12]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 85.94it/s]


Epoch: 1 of 40, 0.034 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 93.17it/s]


Epoch: 2 of 40, 0.008 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 88.47it/s]


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 69.95it/s]


Epoch: 39 of 40, 0.007 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 91.16it/s]


Epoch: 40 of 40, 0.007 min


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,██▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
test_loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,39
lr,0.00129
test_loss,0.01017
train_loss,0.00992


In [2]:
class MyModel(nn.Module):
    def __init__(self, num_features, init_param):
        super(MyModel, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(num_features, init_param),
            nn.BatchNorm1d(init_param),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(init_param, int(init_param/2)),
            nn.BatchNorm1d(int(init_param/2)),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(int(init_param/2), int(init_param/4)),
            nn.BatchNorm1d(int(init_param/4)),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(
            nn.Linear(int(init_param/4), int(init_param/8)),
            nn.BatchNorm1d(int(init_param/8)),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer5 = nn.Linear(int(init_param/8), 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

In [5]:
model = torch.load('../logs/nn_models/classifier/fe_256_1024_60_TEST_MODEL.pth')

In [6]:
model.layer1

Sequential(
  (0): Linear(in_features=60, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.1, inplace=False)
  (3): ReLU()
)